In [1]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import cv2

import config

# configs
data_path = r"runs/detect/exp3/labels"

def inspect():
    str1 = "教室"
    str2 = "课上的抬头人数为："
    path = config.faces_folder
    pic_path = 'source.jpg'
    p = path + '/' + pic_path
    img = cv2.imread(p)
    color = (0, 255, 0)

    grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    classfier = cv2.CascadeClassifier(config.classifier_data)
    faceRects = classfier.detectMultiScale(grey, scaleFactor=1.2, minNeighbors=3, minSize=(32, 32))
    
    print(faceRects)
        
    return faceRects


# get xywh and true resolution x and y

def get_object_xywh(file_name: str) -> list[list[float]]:
    with open(file_name, encoding="utf-8", mode="r") as f:
        file_lines = f.readlines()
        return_list = []
        
        for line in file_lines:
            result = line.split(" ")
            return_list.append([float(num) for num in result[1:]])
            
        return return_list

def get_objects_xywh(directory: str) -> list[list[float]]:
    return_list = []
    
    for root, dirs, files in os.walk(directory):

        for file in files:
            return_list += get_object_xywh(f"{root}/{file}")
    
    return return_list

def get_object_center(xywh_list: list[list[float]]) -> list[tuple[float, float]]:
    return_list = []
    
    for xywh in xywh_list:
        x = xywh[0]
        y = xywh[1]
        w = xywh[2]
        h = xywh[3]
        
        center_x = x + w / 2
        center_y = y + h / 2
        
        return_list.append((center_x, center_y))
    
    return return_list

def get_object_center_scaled(xywh_list) -> list[tuple[int, int]]:
    return_list = []
    
    for xywh in xywh_list:
        x = xywh[0]
        y = xywh[1]
        w = xywh[2]
        h = xywh[3]
        
        center_x = x + w / 2
        center_y = y + h / 2
        
        return_list.append((int(center_x), int(center_y)))
    
    return return_list

rects = inspect()


xywh = get_objects_xywh(data_path)
xywh_df = pd.DataFrame(xywh)

center_data = pd.DataFrame(get_object_center_scaled(rects), columns=["res-x", "res-y"])

plt.figure(figsize=(16, 9))

g = sns.jointplot(data=center_data, x="res-x", y="res-y", xlim=(0, 1), ylim=(1, 0))
g.plot_joint(sns.kdeplot, color="r", zorder=0, levels=6)
g


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
